In [1]:
from mesh import Mesh

In [13]:
#minimal
#filename, quality_index = "Data/simplicial_donut_3.ply", 2
#small
#filename, quality_index = "Data/simplicial_donut_3_deform.ply", 2
#filename, quality_index = "Data/31_r1.00_n4_v256.volume.ply", 3
#filename, quality_index = "Data/31_r1.00_n4_v256.volume_r1.50_n4_v256.volume.ply", 3
filename, quality_index = "Data/31_mean_curvature.ply", 7
#filename, quality_index = "Data/31_min_curvature_meshlab.ply", 7
#medium
#filename, quality_index = "Data/HS_0109_HeiCuBeDa_GigaMesh.ply", 3
#filename, quality_index = "Data/HS_0109_HeiCuBeDa_GigaMesh_mean_curvature.ply", 7
#filename, quality_index = "Data/Messer_Stein/Messer_ohneNachv_GMOCF_r1.50_n4_v256.volume.ply", 3
#large:
#filename, quality_index = "Data/HS_190_HeiCuBeDa_GigaMesh.ply", 3

# Mesh class:
The Mesh class can be initialized using **Mesh()** . It has the following variables and functions:

## Variables:
- **Mesh.filename** filename as string. saved without .ply, but with folder structure

- **Mesh.Vertices** Vertices stored as a dictionary of Vertex objects, keys are Vertex indices

- **Mesh.Edges** Edges stored as a dictionary of Edge objects, keys are numbered

- **Mesh.Faces** Faces stored as a dictionary of Face objects, keys are numbered

- **Mesh.MorseComplex** if not calculated yet: None, otherwise a MorseComplex object with MorseComplex.persistence = 0

- **Mesh.reducedMorseComplexes** a dictionary of MorseComplex objects, that have been reduced by a persistence parameter. The persistence parameter also defines the key of each MorseComplex object

- **Mesh.MorseCells** a dictionary of MorseCells dictionaries. the key for each MorseCell dictionary is given by the persistence of the MorseComplex, these Cells were taken from. Each MorseCell dictionary consits of key, value pairs, where the key gives the label of a cell and the value is a set of vertices that make up this cell.

## Functions:
- **Mesh.load_mesh_ply(filename, quality_index)** loads a .ply file into the Mesh class and takes the value given at quality_index position as the scalar function on the vertices

- **Mesh.info()** prints out Mesh info

- **Mesh.ProcessLowerStars()** calculates the combinatorial gradient and critical simplices; **required for ExtractMorseComplex**

- **Mesh.only_return_ExtractMorseComplex** return MorseComplex without storing it in the Mesh class

- **Mesh.ExtractMorseComplex** calculate MorseComplex and store it in the Mesh class

- **Mesh.only_return_ReducedMorseComplex(persistence)** reduce the MorseComplex based on the given persistence parameter, but dont store it in the Mesh class

- **Mesh.ReducedMorseComplex(persistence)** reduce the MorseComplex based on the given persistence parameter, and store it in the Mesh class

- **Mesh.plot_MorseComplex(MorseComplex, filename, path_color=[255,0,255])** plots the critical simplices of a MorseComplex in red (vertex), green (edge) and blue (face) and the according paths connecting them in magenta. Only writes a ply file containing colored points, so should be used as an overlay for the original mesh

- **Mesh.ExtractMorseCells(MorseComplex)** takes a MorseComplex and gets all connected cells individually that are enclosed by the lines between critical simplices

- **Mesh.plot_MorseCells(persistence, filename)** plots the connected components in each Cell in the same color. Only writes a ply file containing colored points, so should be used as an overlay for the original mesh

In [14]:
data = Mesh()
data.load_mesh_ply(filename, quality_index, inverted=False)
data.info()
data.ProcessLowerStars()
data.ExtractMorseComplex()
data.MorseComplex.info()

Time read data file: 1.2814996000000747
Time read and prepare data: 8.562204399999928
+-------------------------------------------------------
| Mesh Info
+-------------------------------------------------------
| Filename:  Data/31_mean_curvature
| Morse function values range:  [-5.574097, 17.70892]
+-------------------------------------------------------
| Number of Vertices:  146174
| Number of Edges:  438516
| Number of Faces:  292344
+-------------------------------------------------------
| Euler characteristic:  2
+-------------------------------------------------------
Time ProcessLowerStar: 4.198388300000033
Time ExtractMorseComplex and Separatrices: 3.574103100000002
+-------------------------------------------------------
| MorseComplex Info
+-------------------------------------------------------
| Filename:  Data/31_mean_curvature
| Persistence of this Complex:  0
+-------------------------------------------------------
| Number of Vertices:  9805
| Number of Edges:  19670

In [15]:
#data.calculate_BettiNumbers()
#data.plot_PersistenceDiagram()

In [16]:
red_MSC = data.ReduceMorseComplex(45)

Time cancel critical points with  45  persistence:  2.386341399999992
Persistence was higher than the range of function values,
therefore this complex is maximally reduced and can be used for salient edge extraction.


In [17]:
#data.plot_salient_edge_histogram(100, log=True)

In [18]:
#red_MSC.Separatrices.sort(key=lambda x: x[0])
#for pers,sep in red_MSC.Separatrices:
#    print(pers)


In [19]:
#thr = [1.3]
#min_thresh = 10
#max_thresh = 7
#for thresh in thr:
    #Mesh.plot_improved_salient_edge(str(data.filename)+"_improved_salient_edge_justSum"+str(thresh), 
    #                                thresh, min_thresh, max_thresh)
    #data.plot_salient_edge(str(data.filename), thresh)
    #data.plot_salient_edge_pline(str(data.filename), thresh)

In [20]:
# Calculate reduced Complexes for different persistence levels:

persistences = [0.5,1]
for persistence in persistences:
    # reduce morse complex persistence based
    data.ReduceMorseComplex(persistence)
    
    # plot MorseComplexes, calculate MorseCells and plot MorseCells
    reduced_complex = data.reducedMorseComplexes[persistence]
    #reduced_complex.info()
    #data.plot_MorseComplex(reduced_complex, str(data.filename))
    #thresholds = [0.1,0.15,0.2,0.25,0.3]
    #for abs_thresh in thresholds:
    #    data.plot_MorseComplex_thresholded(reduced_complex, 
    #                                       str(data.filename), 
    #                                       abs_thresh)
    #    data.plot_MorseComplex_thresholded_pline(persistence, 
    #                                             str(data.filename), 
    #                                             minimum_length=5, thresh=abs_thresh)
    
    #data.plot_MorseComplex_pline(persistence, str(data.filename))
    data.ExtractMorseCells(reduced_complex)
    #data.write_MorseCellLabels(persistence, str(data.filename))
    #data.plot_MorseCells(persistence, str(data.filename))


Time cancel critical points with  0.5  persistence:  2.2220148999999765
Bd pts 0
Time writing overlay file for bd:  0.337838300000044
Time get MorseCells:  45.93471700000009
Time cancel critical points with  1  persistence:  0.8515756000000465
Bd pts 0
Time writing overlay file for bd:  0.2759421000000657
Time get MorseCells:  14.095173799999998


In [21]:
'''
graph_pers = 4.5
graph = data.GetConnectivityGraph(graph_pers)
graph.remove_small_components(data, persistence=graph_pers, size_thresh=100)
data.write_MorseCellLabels(graph_pers, str(data.filename)+"_error_removed_test2_smallCompRemoved")
graph.remove_weak_edges(data, persistence=graph_pers, absolute_threshold=0.5)
data.write_MorseCellLabels(graph_pers, str(data.filename)+"_error_removed_test2_weakEdgesRemoved")
'''

'\ngraph_pers = 4.5\ngraph = data.GetConnectivityGraph(graph_pers)\ngraph.remove_small_components(data, persistence=graph_pers, size_thresh=100)\ndata.write_MorseCellLabels(graph_pers, str(data.filename)+"_error_removed_test2_smallCompRemoved")\ngraph.remove_weak_edges(data, persistence=graph_pers, absolute_threshold=0.5)\ndata.write_MorseCellLabels(graph_pers, str(data.filename)+"_error_removed_test2_weakEdgesRemoved")\n'

In [22]:

pers = 1
saledge_thresh = 0.8
edge_percentage = 0.35

graph, saledge_pts = data.SalientEdgeConnectivityGraph(saledge_thresh, pers)
for i in range(40):
    if i%5==0:
        print(i,"iterations:",len(graph.conncomps.keys()))
        data.write_MorseCellLabels(pers, 
                                   str(data.filename)+"_"+str(edge_percentage)+"edgePerc_"+str(pers)+
                                   "pers_"+str(saledge_thresh)+"SalEdgeThr_"+str(i)+"_iterations")
    MorseCell = graph.simplify_cells(data.MorseCells[pers], edge_percentage, saledge_pts)


Time getting salient edge points for  0.8 threshold: 0.030426300000044648
Time writing overlay file for bd:  0.25361979999979667
Nb of not found labels:  0
Time writing overlay file for bd:  0.04585740000015903
Time get weighted ConnectivityGraph:  26.696559200000138
0 iterations: 4448
Time writing label txt file: 0.15411140000014711
5 iterations: 971
Time writing label txt file: 0.1575678000001517
10 iterations: 407
Time writing label txt file: 0.15144080000004578
15 iterations: 281
Time writing label txt file: 0.15685979999989286
20 iterations: 229
Time writing label txt file: 0.15778549999981806
25 iterations: 203
Time writing label txt file: 0.15025279999986196
30 iterations: 190
Time writing label txt file: 0.1525890000000345
35 iterations: 180
Time writing label txt file: 0.14861809999979414


In [23]:
graph.remove_small_components(data, pers, size_thresh=300)
print(len(graph.conncomps.keys()))
data.write_MorseCellLabels(pers, str(data.filename)+"_"+str(edge_percentage)+"edgePerc_"+str(pers)+
                                   "pers_"+str(saledge_thresh)+"SalEdgeThr_"+str(i)+"_iterations_small_removed")

12
Time writing label txt file: 0.15448459999993247
